# JP2 to JPEG Conversion Notebook

This notebook demonstrates how to convert JP2 (JPEG 2000) image files to standard JPEG format using Python and ImageMagick.

## Importing necessary libraries

We start by importing the required Python libraries:

In [ ]:
import os
import time
import subprocess
from concurrent.futures import ThreadPoolExecutor
import tqdm  # Optional: for progress bar

Here's what each imported library does:

- `os`: Provides functions for interacting with the operating system
- `time`: Used for measuring the time taken for operations
- `subprocess`: Allows running external commands (like ImageMagick's 'mogrify')
- `ThreadPoolExecutor`: Enables parallel processing of tasks
- `tqdm`: Provides a progress bar for long-running operations

## Main Conversion Script

Now, let's go through the main conversion script step by step:

In [ ]:
# Directory containing .jp2 files
input_dir = './downloads'

# Find all .jp2 files in the directory
jp2_files = []
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.jp2') and not file.startswith('._'):
            jp2_files.append(os.path.join(root, file))

This code does the following:
1. Sets the input directory where the JP2 files are located.
2. Walks through the input directory and its subdirectories.
3. Finds all .jp2 files, excluding those starting with '._' (which are often hidden system files on macOS).
4. Adds the full paths of these files to the `jp2_files` list.

In [ ]:
# Initialize variables
total_time = 0
file_count = len(jp2_files)
error_list = []  # List to record filenames of failed conversions

# Function to convert a single .jp2 file to .jpeg
def convert_and_remove(file_path):
    try:
        start_time = time.time()
        subprocess.run(['mogrify', '-format', 'jpeg', '-quality', '95', file_path], check=True)
        os.remove(file_path)  # Remove original file after successful conversion
        end_time = time.time()
        return end_time - start_time
    except Exception as e:
        error_list.append(file_path)  # Record the failing file path
        return 0  # Return zero time for failed conversions

This section:
1. Initializes variables to keep track of the total time, file count, and any errors.
2. Defines a function `convert_and_remove` that:
   - Uses 'mogrify' (an ImageMagick command) to convert a single file to JPEG format with 95% quality.
   - Removes the original JP2 file if the conversion is successful.
   - Measures and returns the time taken for the conversion.
   - If there's an error, it adds the file to the error list.

In [ ]:
# Use ThreadPoolExecutor to parallelize the file conversion
with ThreadPoolExecutor() as executor:
    # Optional: Use tqdm to show progress bar
    results = list(tqdm.tqdm(executor.map(convert_and_remove, jp2_files), total=file_count))

# Calculate total and average time
total_time = sum(results)
average_time = total_time / file_count if file_count > 0 else 0

# Output results and errors
print(f'Batch conversion complete.')
print(f'Total time: {total_time:.2f} seconds')
print(f'Average time per image: {average_time:.2f} seconds')
if error_list:
    print(f'Failed to convert {len(error_list)} files:')
    for error_file in error_list:
        print(error_file)

This final section:
1. Uses ThreadPoolExecutor to run the conversion function on multiple files in parallel, which can significantly speed up the process on multi-core systems.
2. Uses tqdm to display a progress bar showing how many files have been processed.
3. Calculates and prints the total time taken and the average time per image.
4. If any files failed to convert, it prints the number of failures and lists the files that failed.

This script is designed to efficiently convert a large number of JP2 files to JPEG format, which can be useful for making these images more compatible with common image viewers and web browsers. The use of parallel processing helps to speed up the conversion process when dealing with many files.